# Overview over using `ipyparallel` on Janus

## Using the janus-node profile

In [4]:
from ipyparallel import Client
rcn = Client(profile='janus-node')
print(rcn.ids)

[0, 1]


## Using the janus-core profile

In [5]:
rcc = Client(profile='janus-cpu')
print(rcc.ids)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]


### Monte-Carlo $\pi$

This runs on the notebook server

In [14]:
import random
 
def mc_pi(n):
    count = 0.0
    for i in range(n):
        x = random.random()
        y = random.random()
        if (x**2 + y**2) <= 1:
            count += 1.0
    return count/float(n)*4.


In [16]:
mc_pi(10**7)

3.1417724

### Map and reduce serially

This runs on the notebook server and not on any of the engines of the cluster

In [25]:
from functools import reduce
from operator import add

serial_result = list(map(mc_pi, [10**6, 10**6]))
print(serial_result)
reduce(add, serial_result) / (len(serial_result))

[3.142252, 3.141308]


3.14178

## Running on on of the clusters

### Using a DirectView and then use a synchronous map

Goal:
- Running 24 instances of the `mc_pi` function
- Get the results back in a list and then do a reduce on the list

In [26]:
rcc = Client(profile='janus-cpu')
dview = rcc.direct_view()
presults = dview.map_sync(mc_pi, [10**6]*24)

CompositeError: one or more exceptions from call to method: mc_pi
[Engine Exception]NameError: name 'random' is not defined
[Engine Exception]NameError: name 'random' is not defined
[Engine Exception]NameError: name 'random' is not defined
[Engine Exception]NameError: name 'random' is not defined
.... 20 more exceptions ...

### Error
- Engines are their own python process
- Have not loaded the `random module` into each engine
- The notebook runs on the following crestone node

In [28]:
import socket
hostname = socket.gethostname()
print(hostname)

cnode0105.rc.int.colorado.edu


## Import the module and function to all engines

- Using the `%%px` cell magic
- Show where the engines are running

In [29]:
%%px 
import socket
hostname = socket.gethostname()
print(hostname)

[stdout:0] node0257
[stdout:1] node0257
[stdout:2] node0257
[stdout:3] node0257
[stdout:4] node0257
[stdout:5] node0257
[stdout:6] node0257
[stdout:7] node0257
[stdout:8] node0257
[stdout:9] node0257
[stdout:10] node0257
[stdout:11] node0257
[stdout:12] node0258
[stdout:13] node0258
[stdout:14] node0258
[stdout:15] node0258
[stdout:16] node0258
[stdout:17] node0258
[stdout:18] node0258
[stdout:19] node0258
[stdout:20] node0258
[stdout:21] node0258
[stdout:22] node0258
[stdout:23] node0258


In [30]:
%%px --targets ::2
hostname = socket.gethostname()
print(hostname)

[stdout:0] node0257
[stdout:2] node0257
[stdout:4] node0257
[stdout:6] node0257
[stdout:8] node0257
[stdout:10] node0257
[stdout:12] node0258
[stdout:14] node0258
[stdout:16] node0258
[stdout:18] node0258
[stdout:20] node0258
[stdout:22] node0258


### Let's load the function and the module into the engines

In [31]:
%%px
import random
 
def mc_pi(n):
    count = 0.
    for i in range(n):
        x = random.random()
        y = random.random()
        if (x**2 + y**2) <= 1:
            count += 1.
    return count/n*4.

Let's run the mc_pi function on each of the engines

In [33]:
nengines = len(rcc.ids)
dview = rcc.direct_view()
results = dview.map_sync(mc_pi, [10**6]*nengines)
print(results)

[3.145544, 3.142628, 3.142348, 3.141104, 3.142764, 3.142048, 3.143284, 3.143144, 3.143128, 3.138904, 3.142784, 3.140892, 3.140912, 3.13954, 3.14696, 3.140148, 3.141416, 3.1392, 3.145072, 3.142756, 3.13926, 3.141004, 3.139684, 3.139388]


In [34]:
res_pi = sum(results)/len(results)
print(res_pi)

3.1418296666666663


## Load balanced view

- Use when each function may take different times

In [33]:
lbview = rcc.load_balanced_view()
results = lbview.map_sync(mc_pi, [10**6]*50)
print(results)

[3.143404, 3.142548, 3.138192, 3.14098, 3.143328, 3.141996, 3.140536, 3.143176, 3.141352, 3.141768, 3.140924, 3.138916, 3.140308, 3.141772, 3.141148, 3.142436, 3.142456, 3.142504, 3.139556, 3.14208, 3.140612, 3.14186, 3.143004, 3.1416, 3.140232, 3.142336, 3.140404, 3.142396, 3.140188, 3.14324, 3.140792, 3.14294, 3.140056, 3.142188, 3.1436, 3.141796, 3.142124, 3.1435, 3.1406, 3.139736, 3.141112, 3.137728, 3.140132, 3.139748, 3.1437, 3.137216, 3.143888, 3.1382, 3.14196, 3.140388]


## Simpel element wise parallel computing

In [36]:
import numpy as np
A = np.random.random((128,128))

@dview.parallel(block=True)
def parallel_multiply(A,B):
    return A*B


Serial

In [37]:
C_serial = A*A

In [39]:
C_parallel = parallel_multiply(A,A)

In [40]:
(C_serial==C_parallel).all()

True